In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

Import OSNIR Dataset and shuffle data

In [2]:
# import OSNIR dataset
df = pd.read_csv('../Data/OSNIR_values_extendedv3_new datasetbcsv.csv')
# shuffling rows of OSNIR dataframe and reset indexes
df = df.sample(frac=1)
df.reset_index(inplace=True, drop=True)

Preview of first 5 samples

In [3]:
df.head()

,OSNIRnumerical(dB),Ns,Pch(dBm),L(km),B(GHz),GB(GHz),Nch
0,7.927238,36,3,100,25.0,12.5,9
1,19.307620,5,3,100,25.0,25.0,9
2,15.340119,43,3,10,50.0,0.0,9
3,15.810240,9,0,100,50.0,0.0,9
4,7.259275,41,0,100,12.5,0.0,9


1800 values for training (0,1800-1)\
600 values for validation (1800,2400-1)\
600 values for testing (2400,3000-1)

In [4]:
data = np.array(df)

In [126]:
# input x aka Ns, Pch, L, B, GB, Nch values
x_train = data[0:1800-1, 1:7]
x_valid = data[1800:2400-1, 1:7]
x_test = data[2400:3000-1, 1:7]

# output y aka OSNIR values
y_train = data[0:1800-1, 0]
y_valid = data[1800:2400-1, 0]
y_test = data[2400:3000-1, 0]

In [140]:
x_train

array([[ 36. ,   3. , 100. ,  25. ,  12.5,   9. ],
       [  5. ,   3. , 100. ,  25. ,  25. ,   9. ],
       [ 43. ,   3. ,  10. ,  50. ,   0. ,   9. ],
       ...,
       [ 34. ,   3. ,  50. ,  25. ,  25. ,   9. ],
       [ 25. ,   0. ,  50. ,  25. ,   0. ,  15. ],
       [ 13. ,   0. ,  50. ,  25. ,   0. ,  15. ]])

### Create the Neural Network
1 Input layer, 2 hidden layers (32 neurons each) and 1 Output Layer

In [141]:
model = keras.Sequential([
    keras.layers.Dense(32, input_shape=(6,), activation='relu'),  # first hidden layer
    keras.layers.Dense(32, activation='relu'),  # second hidden layer
    keras.layers.Dense(1, activation='relu') # output layer (3)
])

In [142]:
early_stop = keras.callbacks.EarlyStopping(monitor='loss',mode='min',patience=50)

In [143]:
model.compile(optimizer='adam',
              loss='mse'
              #metrics=['accuracy']
              )

In [144]:
model.fit(x_train, y_train,
          validation_data=(x_valid,y_valid), 
          epochs=500, 
          batch_size=25,
          callbacks=[early_stop])

Epoch 1/500
72/72 [==============================] - 1s 3ms/step - loss: 139.8079 - val_loss: 68.2507
Epoch 2/500
72/72 [==============================] - 0s 2ms/step - loss: 57.9623 - val_loss: 51.0661
Epoch 3/500
72/72 [==============================] - 0s 2ms/step - loss: 45.7559 - val_loss: 43.0968
Epoch 4/500
72/72 [==============================] - 0s 2ms/step - loss: 38.3903 - val_loss: 37.4349
Epoch 5/500
72/72 [==============================] - 0s 2ms/step - loss: 34.1267 - val_loss: 34.5336
Epoch 6/500
72/72 [==============================] - 0s 2ms/step - loss: 31.5742 - val_loss: 34.2358
Epoch 7/500
72/72 [==============================] - 0s 2ms/step - loss: 29.5328 - val_loss: 30.7895
Epoch 8/500
72/72 [==============================] - 0s 2ms/step - loss: 27.9031 - val_loss: 29.6056
Epoch 9/500
72/72 [==============================] - 0s 2ms/step - loss: 26.4295 - val_loss: 28.3174
Epoch 10/500
72/72 [==============================] - 0s 2ms/step - loss: 24.0235 - val_lo

In [209]:
predictions = (model.predict(x_test)) 

19/19 [==============================] - 0s 946us/step


In [305]:
for i in range(10):
    print('%s => %f (expected %f)' % (x_test[i].tolist(), predictions[i], y_test[i] ))

[9.0, -2.0, 100.0, 25.0, 0.0, 9.0] => 16.846992 (expected 16.332073)
[11.0, -10.0, 10.0, 25.0, 0.0, 9.0] => 26.507645 (expected 26.496166)
[2.0, 0.0, 50.0, 25.0, 0.0, 3.0] => 29.163822 (expected 29.275744)
[5.0, -4.0, 50.0, 25.0, 0.0, 3.0] => 26.522964 (expected 26.459889)
[35.0, 0.0, 50.0, 25.0, 0.0, 3.0] => 14.777282 (expected 14.764385)
[37.0, -5.0, 50.0, 25.0, 0.0, 3.0] => 17.350777 (expected 17.640254)
[19.0, -1.0, 10.0, 12.5, 0.0, 9.0] => 18.504673 (expected 18.387800)
[9.0, 0.0, 50.0, 25.0, 0.0, 15.0] => 20.183861 (expected 19.775849)
[10.0, -9.0, 100.0, 25.0, 0.0, 9.0] => 10.270425 (expected 9.873091)
[39.0, -4.0, 100.0, 12.5, 0.0, 9.0] => 10.619940 (expected 10.633879)
